# Steam Sentiment Analysis Projekt - Sarkasmus Erkennung

## Einführung

In diesem Notebook konzentrieren wir uns auf die Integration von **Sarkasmus-Erkennung** in die Sentiment-Analyse von Steam Reviews. Sarkasmus ist ein häufiges Element in Rezensionen, insbesondere im Gaming-Bereich, und kann die Interpretation der Nutzerintention erschweren. Unser Ziel ist es, durch den Einsatz von Sarkasmus-Modellen, den Kontext besser zu erfassen und die Vorhersagegenauigkeit des Modells zu verbessern.

### Warum verwenden wir Sarkasmus-Erkennung?

Gaming-Rezensionen enthalten oft sarkastische Kommentare, die auf den ersten Blick positiv wirken können, jedoch eine negative Nutzererfahrung widerspiegeln. Indem wir Sarkasmus erkennen, können wir die wahre Bedeutung hinter diesen Aussagen besser verstehen und die Klassifikationsleistung für positive und negative Reviews verbessern.

---

## Modelle für die Sarkasmus-Erkennung

Um Sarkasmus zu erkennen, haben wir drei verschiedene Modelle getestet, die jeweils unterschiedliche Ansätze und Trainingsdaten nutzen:

1. **dnzblgn/Sarcasm-Detection-Customer-Reviews**: Trainiert auf Kundenrezensionen. 
- Modell: Fine-tuned version des *microsoft/mdeberta-v3-base* Modells.
- Input und Output: Input: *Text*, Output: *nonsarcasm*, *sarcasm* scores
- [Link](https://huggingface.co/dnzblgn/Sarcasm-Detection-Customer-Reviews)

2. **helinivan/english-sarcasm-detector**: Trainiert mit News Headlines Dataset For Sarcasm Detection von Kaggle.
- Modell: Fine-tuned version des *bert-base-uncased* Modells.
- Input und Output: Input: *Text*, Output: *not_sarcasm*, *sarcasm* (Label) + scores
- [Link](https://huggingface.co/helinivan/english-sarcasm-detector)
3. **mrm8488/t5-base-finetuned-sarcasm-twitter**: Trainiert auf Tweets.
- Modell: Fine-tuned version des *t5-base* Modells von Google.
- Input und Output: Input: *Text*, Output: *sarcasm* (Label)
- [Link](https://huggingface.co/mrm8488/t5-base-finetuned-sarcasm-twitter)

Die entscheidung für die Modelle wurde getroffen, da es kein Sarkasmus Modell existiert, welches speziell für Steam Reviews trainiert wurde. Die Modelle wurden aufgrund ihrer Verfügbarkeit und ihrer Fähigkeit, Sarkasmus in verschiedenen Texten zu erkennen, ausgewählt.

---

### Autoren
- Markus Grau, Daniel Kosma, Mikail Yildiz

## Libraries

In [1]:
import pandas as pd
from tqdm import tqdm
pd.set_option('display.float_format', '{:.4f}'.format) #runden der zahlen 

from transformers import AutoTokenizer, AutoModelForSequenceClassification # Tokenizer und Modell Classification für BERT und Co.
from transformers import T5Tokenizer, T5ForConditionalGeneration # Tokenizer und Modell für T5

from torch.utils.data import DataLoader, Dataset # PyTorch, für Neural Networks
import torch # PyTorch

from sklearn.model_selection import train_test_split # Train-Test Split

In [2]:
df = pd.read_parquet("data/processed_data/steam_reviews_filtered.parquet") # daten einlesen 

In [3]:
df.head(5)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,publisher,genres,notes,cleaned_review,token_length
269238,620980,Beat Saber,79244667,english,"Since I am 80 + years old, it is very importan...",2020-11-14 10:51:22,2020-11-14 10:51:22,True,1493,176,...,1,2544.0000,0.0000,2451.0000,2020-12-30 09:08:22,Beat Games,[Indie],None,"since i am 80 + years old, it is very importan...",52
1776430,1113000,Persona 4 Golden,70806847,english,Please buy this game if you want more Persona ...,2020-06-15 02:03:47,2020-11-26 04:15:44,True,1490,31,...,12,9612.0000,0.0000,499.0000,2020-11-26 04:08:17,SEGA,[RPG],Alcohol Reference\r\nAnimated Blood\r\nLanguag...,please buy this game if you want more persona ...,18
1396365,1145360,Hades,75662801,english,You can date the medusa head. Post Launch Edi...,2020-09-08 19:18:50,2020-10-10 19:57:02,True,1486,745,...,7,3965.0000,0.0000,2445.0000,2020-12-16 12:52:51,Supergiant Games,"[Action, Indie, RPG]",None,you can date the medusa head.\n\npost launch e...,21
354159,1225330,NBA 2K21,75410143,english,There is very little difference from 2k20. The...,2020-09-04 06:20:35,2020-09-04 06:20:35,False,1484,99,...,1,698.0000,576.0000,121.0000,2021-01-15 00:10:47,2K,"[Simulation, Sports]",None,there is very little difference from 2k20. the...,79
2574101,105600,Terraria,78393147,english,---{Graphics}--- ✖ Masterpiece ✖ Beautiful ✅Go...,2020-10-30 12:14:22,2020-11-26 04:01:45,True,1461,156,...,13,2406.0000,0.0000,2406.0000,2020-10-24 08:38:17,None,None,None,---{graphics}---\n✖ masterpiece\n✖ beautiful\n...,459


## Vorbereitung
Da unser Datensatz sehr groß ist, und Transformer bei der Verarbeitung von großen Datensätzen sehr langsam ist, haben wir uns entschieden, den Datensatz für den Vergleich zwischen den Modellen auf 10% zu reduzieren. 

In [4]:
df_stratified, _ = train_test_split(df, test_size=0.9, stratify=df['recommended'], random_state=42) #statify wird benutzt um die Daten gleichmäßig aufzuteilen

In [5]:
df_stratified['recommended'].value_counts()

recommended
True     27916
False     7915
Name: count, dtype: int64

### Einlesen der GPU
Zuerst müssen wir die GPU aktivieren, um die Modelle schneller zu trainieren und zu testen. Dafür wurden handelsübliche GPUs der Marke NVIDIA verwendet.

In [6]:
# Prüfe, ob eine GPU verfügbar ist
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU ist verfügbar: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Keine GPU verfügbar, CPU wird verwendet")

GPU ist verfügbar: NVIDIA GeForce RTX 2080 SUPER


### Funktionen und Settings
Um die Modelle effizient zu testen, werden spezifische Funktionen definiert, die für die Vorhersage und Auswertung der Modelle erforderlich sind.

#### DataLoader Klasse
Es wird eine Klasse für einen DataLoader erstellt, die auf die Modelle zugeschnitten ist. Diese Klasse übernimmt das Tokenisieren der Texte und bereitet sie für die Eingabe in die Modelle vor.

#### Vorhersagefunktion
Eine generische Funktion wird definiert, die mit dem gegebenen Modell und den gegebenen Texten arbeitet, um Sarkasmus-Vorhersagen zu treffen. Diese Funktion verschiebt die Eingabedaten auf die GPU, führt das Modell aus und wandelt die Logits in Wahrscheinlichkeiten um.

#### Batch-Verarbeitung
Eine weitere Funktion teilt die Daten in Batches auf, um die Vorhersage zu beschleunigen. Diese Funktion erstellt einen DataLoader, verarbeitet die Daten in Batches und berechnet die Sarkasmus-Scores.


*Das T5 Modell hat eine andere Vorhersagefunktion, da es ein Text-to-Text Modell ist. Es erhält eine Liste von Strings und gibt eine Liste von Strings zurück und muss deswegen anders behandelt werden.*

In [7]:
# Klasse für den Data Loader erstellen
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_length=512):
        self.reviews = reviews # text das rein geht
        self.tokenizer = tokenizer # tokenizer model
        self.max_length = max_length # maximale länge des textes

    def __len__(self):
        return len(self.reviews) # länge des textes returnen

    def __getitem__(self, idx):
        review = self.reviews.iloc[idx]['cleaned_review'] # review auslesen
        inputs = self.tokenizer(review, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt") # tokenizing aktivieren
        inputs = {key: value.squeeze(0) for key, value in inputs.items()}  # entfernung von unnötigen Dimensionen
        return inputs, self.reviews.iloc[idx]['review_id']

# Generische Funktion, um Sarkasmus im Batch zu erkennen
def predict_sarcasm_batch(batch, model):
    """Funktion evaluiert ein Modell auf einem Batch von Daten und gibt die Sarkasmus-Wahrscheinlichkeiten zurück.

    Args:
        batch (torch.Dataloader): DataLoader Instanz der Eingabedaten im gegebenen Batch
        model (torch.AutoModelForSequenceClassification): Torch Modell für die Klassifikation

    Returns:
        list: Liste mit den Warscheinlichkeiten für Sarkasmus
    """
    inputs = {key: val.to(device) for key, val in batch.items()}  # Batch auf die GPU verschieben
    outputs = model(**inputs)  # Modell-Durchlauf für den Batch
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)  # Logits in Wahrscheinlichkeiten umwandeln
    return scores[:, 1].tolist()  # Extrahiere der warscheinlichkeiten

# Funktion zur Verarbeitung des gesamten DataFrames in Batches
def process_sarcasm_scores_with_dataloader(df, model, tokenizer, batch_size=4):
    """Funktion zur Verarbeitung des gesamten DataFrames in Batches und Berechnung der Sarkasmus-Scores.

    Args:
        df (PandasDataFrame): Ein Pandas DataFrame mit den Daten für die Verarbeitung
        model (torch.AutoModelForSequenceClassification): Torch Modell für die Klassifikation
        tokenizer (torch.Autotokenizer): Tokernizer für die Verarbeitung der Daten
        batch_size (int, optional): Die größe der Batches die Parallel auf die GPU verschoben werden. Defaultwert ist 4.

    Returns:
        df: Pandas DataFrame mit 'review_id' und 'sarcasm_score' Spalten
    """
    sarcasm_scores = []
    review_ids = list(df['review_id']) # liste der ids erstellen

    # Dataset erstellen und mit DataLoader verbinden
    dataset = ReviewDataset(df, tokenizer) 
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Sarkasmus-Scores in Batches berechnen
    for batch, ids in tqdm(dataloader, desc="Verarbeite Batches"):
        batch_scores = predict_sarcasm_batch(batch, model)
        sarcasm_scores.extend(batch_scores)
    

    return pd.DataFrame({'review_id': review_ids, 'sarcasm_score': sarcasm_scores})

### Testbeispiele

Hier testen wir zuerst einige Beispieltexte aus dem Internet, einige sind Sarkastisch andere nicht, kommt immer drauf an wers liest ;D.

In [8]:
# einige beispiele
sarcastic_reviews = [
    "I want to refund this apple", 
    "I love this game so much, that I stopped playing it after 5 minutes", 
    "This game is so bad, it's almost good", 
    "Researchers have found out, that being sarcastic is sarcastically not sarcastic.",
    "Oh great, another Monday! Just what I needed.",
    "This is the best burnt toast I have ever had!",
    "I bought this shirt two sizes too small, but at least it fits my cat now.",
    "Sure, because I love waiting in traffic for hours!",
    "I totally believe the internet never lies.",
    "Oh fantastic, the printer broke right when I needed it most!",
    "Yes, please give me more bugs in this software. It's what I live for!",
    "Just what I needed, another useless email in my inbox.",
    "Oh perfect, I was really hoping for rain on my day off!",
    "This vacuum cleaner is so good, it sucks up all my free time as well.",
    "Oh wonderful, I can't wait to clean the house on a Friday night!"
]
test_ids = range(1, len(sarcastic_reviews) + 1)

df_test_sarcasm = pd.DataFrame({"cleaned_review": sarcastic_reviews,"review_id": test_ids})
df_test_sarcasm

,cleaned_review,review_id
0,I want to refund this apple,1
1,"I love this game so much, that I stopped playi...",2
2,"This game is so bad, it's almost good",3
3,"Researchers have found out, that being sarcast...",4
4,"Oh great, another Monday! Just what I needed.",5
5,This is the best burnt toast I have ever had!,6
6,"I bought this shirt two sizes too small, but a...",7
7,"Sure, because I love waiting in traffic for ho...",8
8,I totally believe the internet never lies.,9
9,"Oh fantastic, the printer broke right when I n...",10


## Sarkasmusmodelle

### dnzblgn/Sarcasm-Detection-Customer-Reviews

Das Modell ist eine feinabgestimmte Version von ```microsoft/mdeberta-v3-base```, die speziell für Sarkasmuserkennung in Kundenbewertungen entwickelt wurde. 

Es klassifiziert Texte als sarkastisch oder nicht sarkastisch und unterstützt die Analyse von vermeintlich positiven Kommentaren, um versteckte Kritik oder Ironie zu erkennen. 

Das Modell eignet sich besonders gut zur Ergänzung von Sentimentanalyse, indem es Sarkasmus in positiven Bewertungen identifiziert.

Dies werden wir nun anhand ein paar Beispielen testen und dann für die 10% der Reviews durchlaufen lassen.

In [9]:
# Modelle laden
tokenizer = AutoTokenizer.from_pretrained("dnzblgn/Sarcasm-Detection-Customer-Reviews") # Tokenizer des Modells definieren
model = AutoModelForSequenceClassification.from_pretrained("dnzblgn/Sarcasm-Detection-Customer-Reviews").to(device) #Modell in die GPU laden

In [10]:
test_resuts_sdcr = process_sarcasm_scores_with_dataloader(df_test_sarcasm, model, tokenizer, batch_size=4)  

Verarbeite Batches: 100%|██████████| 4/4 [00:01<00:00,  2.73it/s]


In [11]:
pd.merge(df_test_sarcasm, test_resuts_sdcr, on='review_id')

,cleaned_review,review_id,sarcasm_score
0,I want to refund this apple,1,0.0000
1,"I love this game so much, that I stopped playi...",2,0.0000
2,"This game is so bad, it's almost good",3,0.9998
3,"Researchers have found out, that being sarcast...",4,1.0000
4,"Oh great, another Monday! Just what I needed.",5,1.0000
5,This is the best burnt toast I have ever had!,6,0.0001
6,"I bought this shirt two sizes too small, but a...",7,1.0000
7,"Sure, because I love waiting in traffic for ho...",8,1.0000
8,I totally believe the internet never lies.,9,0.9985
9,"Oh fantastic, the printer broke right when I n...",10,1.0000


Wir sehen, das es bei den Beispielen gut funktioniert, und werden nun die reviews durchlaufen lassen.

### Durchlaufen mit 10% der Reviews

Wir lassen es nun mit den 10% der Reviews durchlaufen, speichern die ergebnisse in einem parquet ab und schauen und die Ergebnisse für die 10% an, wo Sarkasmus erkannt wurde.

In [12]:
df_sarcasm_scores_sdcr = process_sarcasm_scores_with_dataloader(df_stratified, model, tokenizer, batch_size=4) #sdcr = Sarcasm Detection Customer Reviews

Verarbeite Batches: 100%|██████████| 8958/8958 [15:46<00:00,  9.46it/s]


In [13]:
df_merged_sdcr = pd.merge(df_stratified, df_sarcasm_scores_sdcr, on='review_id') #merge der beiden dataframes, um nachzuverfolgen

In [14]:
df_merged_sdcr.head(5)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,publisher,genres,notes,cleaned_review,token_length,sarcasm_score
0,107410,Arma 3,70165190,english,Extremely in depth play for military tactics. ...,2020-06-01 21:45:47,2020-06-01 21:45:47,True,1,0,...,3388.0000,0.0000,343.0000,2020-08-26 03:57:06,None,None,None,extremely in depth play for military tactics. ...,64,0.0000
1,1145360,Hades,85003471,english,A fantastic rouge-lite that has enough variety...,2021-01-19 23:42:57,2021-01-19 23:42:57,True,1,0,...,5352.0000,6.0000,5351.0000,2021-01-20 00:11:54,Supergiant Games,"[Action, Indie, RPG]",None,a fantastic rouge-lite that has enough variety...,59,0.0000
2,683320,GRIS,79658641,english,"I like artsy little indy games, and this one t...",2020-11-22 13:11:17,2020-11-22 13:11:17,False,3,1,...,84.0000,0.0000,84.0000,2020-11-22 12:58:52,Devolver Digital,"[Adventure, Indie]",None,"i like artsy little indy games, and this one t...",262,0.0000
3,814380,Sekiro™: Shadows Die Twice,61385460,english,hard game veri good cool animations i lika ver...,2020-01-05 22:43:23,2020-01-05 22:43:23,True,0,1,...,1871.0000,0.0000,1169.0000,2020-11-19 11:57:47,Activision (Excluding Japan and Asia),"[Action, Adventure]",None,hard game veri good cool animations i lika ver...,35,0.0000
4,374320,DARK SOULS™ III,64935286,english,"very good, anyone who say its bad says so beca...",2020-03-12 23:31:19,2020-03-12 23:31:19,True,3,0,...,9444.0000,0.0000,4652.0000,2020-12-08 16:38:00,"FromSoftware, Inc.",[Action],None,"very good, anyone who say its bad says so beca...",17,0.0000


In [15]:
df_merged_sdcr.to_parquet("data/sarcasm_results/sarcasm_scores_sdcr.parquet") # abspeichern der Daten

In [21]:
checkup_sdcr = df_merged_sdcr[["app_name","cleaned_review","sarcasm_score","recommended"]]
results = checkup_sdcr.sort_values(by='sarcasm_score', ascending=False).head(10)
results

,app_name,cleaned_review,sarcasm_score,recommended
26974,Total War: WARHAMMER II,this game is an absolute masterpiece of strate...,1.0000,True
23623,Tomb Raider,this game makes me want to die more than usual.,1.0000,False
9868,Hollow Knight,this is a masterpiece of a video game that set...,1.0000,True
19649,MORDHAU,this game makes me want to set fire to governm...,1.0000,False
9888,DARK SOULS™ III,this game made me lose all faith in humanity.,1.0000,False
34153,DOOM Eternal,this game is the best kind of suffering possible.,1.0000,True
28458,Warhammer: Vermintide 2,residentsleeper game; makes me wanna fall asleep.,1.0000,False
17953,Sekiro™: Shadows Die Twice,this is a shitty fucking game made by people t...,1.0000,False
11094,Rust,this game is the equivalent of gap-toothed bum...,1.0000,False
27545,Sid Meier's Civilization V,bodypillowism will live forever in the hearts ...,1.0000,True


In [22]:
results["cleaned_review"].values

array(['this game is an absolute masterpiece of strategy games.',
       'this game makes me want to die more than usual.',
       'this is a masterpiece of a video game that sets impossible standards for all indie games to come.',
       'this game makes me want to set fire to government buildings.',
       'this game made me lose all faith in humanity.',
       'this game is the best kind of suffering possible.',
       'residentsleeper game; makes me wanna fall asleep.',
       'this is a shitty fucking game made by people that hate themselves.',
       'this game is the equivalent of gap-toothed bumpkins kicking each other in the crotch for entertainment.',
       'bodypillowism will live forever in the hearts of all that believe it.'],
      dtype=object)

Wenn wir uns die Top 10 der erkannten Sarkasmus Reviews anschauen, sehen wir das die meisten davon eher nicht nach Sarkasmus ausschauen.

## helinivan/english-sarcasm-detector
Der English Sarcasm Detector ist ein Textklassifikationsmodell, das entwickelt wurde, um Sarkasmus in den Titeln von Nachrichtenartikeln zu erkennen. 

Es basiert auf dem BERT-base-uncased Modell und wurde mit einem Datensatz aus [Kaggle News Headlines Dataset for Sarcasm Detection](https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection) trainiert. 

Das Modell zielt darauf ab, Sarkasmus anhand der Struktur und des Tons in den Schlagzeilen von Nachrichtenartikeln zu erkennen.

In [23]:
# Lade das Sarkasmus-Modell und den Tokenizer
tokenizer = AutoTokenizer.from_pretrained("helinivan/english-sarcasm-detector") # Tokenizer des Modells definieren
model = AutoModelForSequenceClassification.from_pretrained("helinivan/english-sarcasm-detector").to(device) #Modell in die GPU laden

In [24]:
test_resuts_helinivan = process_sarcasm_scores_with_dataloader(df_test_sarcasm, model, tokenizer, batch_size=4)  

Verarbeite Batches:   0%|          | 0/4 [00:00<?, ?it/s]c:\Users\mikai\anaconda3\envs\private_env\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Verarbeite Batches: 100%|██████████| 4/4 [00:00<00:00, 15.11it/s]


*Das Userwarning kann ignoriert werden, da es nur eine Warnung ist, weil keine FlashAttention gegeben ist in der Torch version.*

In [25]:
pd.merge(df_test_sarcasm, test_resuts_helinivan, on='review_id')

,cleaned_review,review_id,sarcasm_score
0,I want to refund this apple,1,0.0107
1,"I love this game so much, that I stopped playi...",2,0.0176
2,"This game is so bad, it's almost good",3,0.0045
3,"Researchers have found out, that being sarcast...",4,0.6446
4,"Oh great, another Monday! Just what I needed.",5,0.0795
5,This is the best burnt toast I have ever had!,6,0.0072
6,"I bought this shirt two sizes too small, but a...",7,0.0107
7,"Sure, because I love waiting in traffic for ho...",8,0.0300
8,I totally believe the internet never lies.,9,0.0106
9,"Oh fantastic, the printer broke right when I n...",10,0.0219


Dieses Modell schwächelt gegenüber das andere modell mehr. Es erkennt Sarkasmus nicht so gut, wie das andere Modell, außer für das eine Beispiel, welches wie ein Nachrichtentitel aussieht.

Dennoch wird das Model nun mit den 10% der Reviews durchlaufen und die Ergebnisse gespeichert, auch wenn es nicht zum Use-Case passt.

In [26]:
df_sarcasm_scores_helinivan = process_sarcasm_scores_with_dataloader(df_stratified, model, tokenizer, batch_size=2)  

Verarbeite Batches: 100%|██████████| 17916/17916 [09:01<00:00, 33.08it/s]


In [27]:
df_merged_helinivan = pd.merge(df_stratified, df_sarcasm_scores_helinivan, on='review_id')

In [28]:
df_merged_helinivan.head(5)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,publisher,genres,notes,cleaned_review,token_length,sarcasm_score
0,107410,Arma 3,70165190,english,Extremely in depth play for military tactics. ...,2020-06-01 21:45:47,2020-06-01 21:45:47,True,1,0,...,3388.0000,0.0000,343.0000,2020-08-26 03:57:06,None,None,None,extremely in depth play for military tactics. ...,64,0.0139
1,1145360,Hades,85003471,english,A fantastic rouge-lite that has enough variety...,2021-01-19 23:42:57,2021-01-19 23:42:57,True,1,0,...,5352.0000,6.0000,5351.0000,2021-01-20 00:11:54,Supergiant Games,"[Action, Indie, RPG]",None,a fantastic rouge-lite that has enough variety...,59,0.0070
2,683320,GRIS,79658641,english,"I like artsy little indy games, and this one t...",2020-11-22 13:11:17,2020-11-22 13:11:17,False,3,1,...,84.0000,0.0000,84.0000,2020-11-22 12:58:52,Devolver Digital,"[Adventure, Indie]",None,"i like artsy little indy games, and this one t...",262,0.0073
3,814380,Sekiro™: Shadows Die Twice,61385460,english,hard game veri good cool animations i lika ver...,2020-01-05 22:43:23,2020-01-05 22:43:23,True,0,1,...,1871.0000,0.0000,1169.0000,2020-11-19 11:57:47,Activision (Excluding Japan and Asia),"[Action, Adventure]",None,hard game veri good cool animations i lika ver...,35,0.0411
4,374320,DARK SOULS™ III,64935286,english,"very good, anyone who say its bad says so beca...",2020-03-12 23:31:19,2020-03-12 23:31:19,True,3,0,...,9444.0000,0.0000,4652.0000,2020-12-08 16:38:00,"FromSoftware, Inc.",[Action],None,"very good, anyone who say its bad says so beca...",17,0.8143


In [29]:
df_merged_helinivan.to_parquet("data/sarcasm_results/sarcasm_scores_helinivan.parquet")

In [30]:
checkup_helinivan = df_merged_helinivan[["app_name","cleaned_review","sarcasm_score","recommended"]]
results2 = checkup_helinivan.sort_values(by='sarcasm_score', ascending=False).head(10)
results2

,app_name,cleaned_review,sarcasm_score,recommended
30152,Rise of the Tomb Raider,"good game, 50% person who owned this actually ...",0.9868,True
4749,Raft,not enough enough content just grind absolutel...,0.9863,False
8246,Stardew Valley,its good for someone who has alot of time on t...,0.9836,False
15019,Warhammer: Vermintide 2,no idea whats going on just love smashing rats...,0.9834,True
9787,The Forest,first time getting chased down by naked people...,0.9822,True
11668,Bloons TD 6,stinky h*man cant pop bloon like monke can,0.9811,True
23953,Rust,shit game can't even get out of spawn without ...,0.9809,False
23043,Arma 3,epic game not really hard to get into just got...,0.9796,True
18355,People Playground,"very fun sandbox, syringes kinda hard to injec...",0.9784,True
24188,Helltaker,game no good for one with pea brain like me\nn...,0.9782,False


In [31]:
results2["cleaned_review"].values

array(['good game, 50% person who owned this actually finish the game, if anyone wondering that actually good number.',
       'not enough enough content just grind absolutely worthless for that amount of money',
       "its good for someone who has alot of time on their hands and patience but with someone who doesn't really have alot of time to use its not that fun since its basically the same thing over and over again also plant water harvest repeat",
       'no idea whats going on just love smashing rats heads',
       'first time getting chased down by naked people more fun to beet them up with friends to put some clothes on i recommend',
       'stinky h*man cant pop bloon like monke can',
       "shit game can't even get out of spawn without some no life killing me",
       'epic game not really hard to get into just gotta know what servers to join highly reccomended\n',
       'very fun sandbox, syringes kinda hard to inject so put next to person and shoot to release cloud of ga

Wir sehen wieder das gleiche Resultat wie vorhin, das es im Falle der Steam Reviews den Sarkasmus nicht so gut erkennt. Deswegen wird es nicht weiter verwendet.

### mrm8488/t5-base-finetuned-sarcasm-twitter
Das Modell ist eine feinabgestimmte Version des ```t5-base``` Modells von Google, das speziell für die Erkennung von Sarkasmus in Tweets entwickelt wurde. 

Es klassifiziert Texte als sarkastisch oder nicht sarkastisch und unterstützt die Analyse von kurzen, prägnanten Kommentaren, um versteckte Kritik oder Ironie zu erkennen.

Das Modell eignet sich besonders gut zur Ergänzung von Sentimentanalyse, indem es Sarkasmus in kurzen Texten identifiziert.

*Das Modell ist ein Text zu Text Modell, und deswegen wird die Vorhersagefunktion anders behandelt!*

In [32]:
# Custom Dataset für die Verwendung mit DataLoader, AUFBAU WIE OBEN!
class ReviewDatasetT5(Dataset):
    def __init__(self, reviews, tokenizer, max_length=512):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews.iloc[idx]['cleaned_review']
        inputs = self.tokenizer.encode_plus(
            review, return_tensors="pt", padding='max_length', truncation=True, max_length=self.max_length
        )
        inputs = {key: value.squeeze(0) for key, value in inputs.items()}  
        return inputs, self.reviews.iloc[idx]['review_id']

# Generische Funktion, um Sarkasmus im Batch zu erkennen
def predict_sarcasm_t5_batch(batch, model, tokenizer):
    """Eine Funktion zur Vorhersage von Sarkasmus in einem Batch von Daten unter Verwendung von T5.

    Args:
        batch (torch.Dataloader): DataLoader Instanz der Eingabedaten im gegebenen Batch
        model (torch.T5ForConditionalGeneration): T5 Modell für die Generierung von Text
        tokenizer (torch.T5Tokenizer): T5 Tokenizer für die Verarbeitung der Daten

    Returns:
        list: Liste mit den Sarkasmus-Vorhersagen
    """
    inputs = {key: val.to(device) for key, val in batch.items()}  # Transferiere den Batch auf die GPU
    output_sequences = model.generate(input_ids=inputs['input_ids'], max_length=3) # T5 generiert die Ausgabe, max_length=3 für Sarkasmus ist wichtig da es ein Text to Text Modell ist!!!!!!!!!!!!!!!!!!!!!!
    predictions = [tokenizer.decode(seq, skip_special_tokens=True) for seq in output_sequences]
    return predictions

# Funktion zur Verarbeitung des gesamten DataFrames in Batches
def process_sarcasm_scores_t5_with_dataloader(df, model, tokenizer, batch_size=4):
    """Funktion zur Verarbeitung des gesamten DataFrames in Batches und Berechnung der Sarkasmus-Scores.
    
    Args:
        df (PandasDataFrame): Pandas DataFrame mit den Daten für die Verarbeitung
        model (torch.T5ForConditionalGeneration): Torch Modell für die Generierung von Text
        tokenizer (torch.T5Tokenizer): T5 Tokenizer für die Verarbeitung der Daten
        batch_size (int, optional): Die größe der Batches die Parallel auf die GPU verschoben werden. Defaultwert ist 4.

    Returns:
        df: Pandas DataFrame mit 'review_id' und 'sarcasm_prediction' Spalten
    """
    sarcasm_scores = []
    review_ids = list(df['review_id'])

    # Dataset und DataLoader 
    dataset = ReviewDatasetT5(df, tokenizer) 
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Berechne Sarkasmus-Scores in Batches
    for batch, ids in tqdm(dataloader, desc="Verarbeite Batches"):
        batch_predictions = predict_sarcasm_t5_batch(batch, model, tokenizer)
        sarcasm_scores.extend(batch_predictions)
    
    # DataFrame erstellen mit 'review_id' und 'sarcasm_prediction'
    return pd.DataFrame({'review_id': review_ids, 'sarcasm_prediction': sarcasm_scores})



In [33]:
# Hauptteil: Modell und Tokenizer laden
model_name = "mrm8488/t5-base-finetuned-sarcasm-twitter"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [34]:
df_sarcasm_predictions_t5 = process_sarcasm_scores_t5_with_dataloader(df_test_sarcasm, model, tokenizer, batch_size=4)

Verarbeite Batches: 100%|██████████| 4/4 [00:00<00:00,  4.58it/s]


In [35]:
pd.merge(df_test_sarcasm, df_sarcasm_predictions_t5, on='review_id')

,cleaned_review,review_id,sarcasm_prediction
0,I want to refund this apple,1,normal
1,"I love this game so much, that I stopped playi...",2,normal
2,"This game is so bad, it's almost good",3,derison
3,"Researchers have found out, that being sarcast...",4,derison
4,"Oh great, another Monday! Just what I needed.",5,normal
5,This is the best burnt toast I have ever had!,6,normal
6,"I bought this shirt two sizes too small, but a...",7,normal
7,"Sure, because I love waiting in traffic for ho...",8,normal
8,I totally believe the internet never lies.,9,derison
9,"Oh fantastic, the printer broke right when I n...",10,normal


In [36]:
df_sarcasm_predictions_t5 = process_sarcasm_scores_t5_with_dataloader(df_stratified, model, tokenizer, batch_size=4)

Verarbeite Batches: 100%|██████████| 8958/8958 [25:21<00:00,  5.89it/s]


In [37]:
t5_merge = pd.merge(df_stratified, df_sarcasm_predictions_t5, on='review_id')

In [38]:
t5_merge.head(5)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,publisher,genres,notes,cleaned_review,token_length,sarcasm_prediction
0,107410,Arma 3,70165190,english,Extremely in depth play for military tactics. ...,2020-06-01 21:45:47,2020-06-01 21:45:47,True,1,0,...,3388.0000,0.0000,343.0000,2020-08-26 03:57:06,None,None,None,extremely in depth play for military tactics. ...,64,normal
1,1145360,Hades,85003471,english,A fantastic rouge-lite that has enough variety...,2021-01-19 23:42:57,2021-01-19 23:42:57,True,1,0,...,5352.0000,6.0000,5351.0000,2021-01-20 00:11:54,Supergiant Games,"[Action, Indie, RPG]",None,a fantastic rouge-lite that has enough variety...,59,normal
2,683320,GRIS,79658641,english,"I like artsy little indy games, and this one t...",2020-11-22 13:11:17,2020-11-22 13:11:17,False,3,1,...,84.0000,0.0000,84.0000,2020-11-22 12:58:52,Devolver Digital,"[Adventure, Indie]",None,"i like artsy little indy games, and this one t...",262,normal
3,814380,Sekiro™: Shadows Die Twice,61385460,english,hard game veri good cool animations i lika ver...,2020-01-05 22:43:23,2020-01-05 22:43:23,True,0,1,...,1871.0000,0.0000,1169.0000,2020-11-19 11:57:47,Activision (Excluding Japan and Asia),"[Action, Adventure]",None,hard game veri good cool animations i lika ver...,35,normal
4,374320,DARK SOULS™ III,64935286,english,"very good, anyone who say its bad says so beca...",2020-03-12 23:31:19,2020-03-12 23:31:19,True,3,0,...,9444.0000,0.0000,4652.0000,2020-12-08 16:38:00,"FromSoftware, Inc.",[Action],None,"very good, anyone who say its bad says so beca...",17,derison


In [39]:
t5_merge.to_parquet("data/sarcasm_results/sarcasm_predictions_t5.parquet")

In [61]:
checkup_t5 = t5_merge[["app_name","cleaned_review","sarcasm_prediction","recommended"]]

In [65]:
#hole 10 mit derision und 10 mit normal und schaue ob die mit derision wirklich sarkastisch sind
results3_derision = checkup_t5[checkup_t5["sarcasm_prediction"].str.contains("derison")].head(10)
results3_normal = checkup_t5[checkup_t5["sarcasm_prediction"].str.contains("normal")].head(10)
results3 =  pd.concat([results3_derision,results3_normal],axis=0,ignore_index=True)

In [67]:
results3

,app_name,cleaned_review,sarcasm_prediction,recommended
0,DARK SOULS™ III,"very good, anyone who say its bad says so beca...",derison,True
1,Yakuza 0,"i say: ""buy the yakuza game!"" and ""buy all yak...",derison,True
2,Saints Row: The Third,you can't complain about this game :p,derison,True
3,Tomb Raider,one of the most boring pieces of shit i've pla...,derison,False
4,BATTLETECH,barely playable on an i3. slow paced combat. t...,derison,False
5,DOOM Eternal,the bethesda.net account issues ruin this game...,derison,False
6,Among Us,"[b]""you are the impostor!""[/b]\n[b]""nooo yoou ...",derison,True
7,Among Us,ah yes. let's all spawn in the game and do the...,derison,True
8,Grand Theft Auto V,go to youtube before buyng it !!!!\n,derison,True
9,Arma 3,this game is too complicated for my smooth lit...,derison,True


In [68]:
results3["cleaned_review"].values

array(['very good, anyone who say its bad says so because they arnt good at it',
       'i say: "buy the yakuza game!" and "buy all yakuza games!"',
       "you can't complain about this game :p",
       "one of the most boring pieces of shit i've played in my life.",
       'barely playable on an i3. slow paced combat. travelling  and long hold times takes up most of your time, mechcommander is way much more better.',
       "the bethesda.net account issues ruin this game before you even get to play it. \n\ni already have a bethesda.net account, it wouldn't link to my steam account, (which is a requirement) because bethesda.net linked my steam account to an email address that didn't even have a bethesda.net account. \n\nwaiting on bethesda support to (possibly) unlink my steam account. \n\nwas hyped for this game for a year and now it's tainted by something that could have been avoided as easily as allowing you to change the bethesda.net account in game, instead of having to do accoun

Wie man erkennen kann, erkennt das Modell Sarkasmus in videospielen auch wieder nicht so gut, wie das erste Modell.

## Vergleichen der Modelle
Nun werden alle 3 Modelle mit den 10% der Reviews durchgelaufen sind gegeneinander verglichen.

Wir schauen uns dafür an, wie oft die Modelle das gleiche Review als Sarkastisch erkannt haben, und wie oft sie sich unterscheiden.

In [76]:
# Merge alle ergebnisse der Modelle miteinander
merged_pd = pd.merge(df_sarcasm_scores_sdcr, df_sarcasm_scores_helinivan, on='review_id', suffixes=('_sdcr', '_helinivan'))
merged_pd = pd.merge(merged_pd, df_sarcasm_predictions_t5, on='review_id')
merged_pd = pd.merge(merged_pd, df_stratified, on='review_id')

# renaming für bessere übersicht
merged_pd.rename(columns={
    'sarcasm_score_sdcr': 'sarcasm_score_sdcr',
    'sarcasm_score_helinivan': 'sarcasm_score_helinivan',
    'sarcasm_prediction': 'sarcasm_prediction_t5'
}, inplace=True)

# alles aauf normal oder derision setzen
merged_pd['sarcasm_score_sdcr'] = merged_pd['sarcasm_score_sdcr'].apply(lambda x: 'derison' if x > 0.5 else 'normal')
merged_pd['sarcasm_score_helinivan'] = merged_pd['sarcasm_score_helinivan'].apply(lambda x: 'derison' if x > 0.5 else 'normal')

In [77]:
# alles raushauen was nicht derison oder normal ist
merged_pd = merged_pd[(merged_pd['sarcasm_score_sdcr'] == 'derison') | (merged_pd['sarcasm_score_sdcr'] == 'normal')]
merged_pd = merged_pd[(merged_pd['sarcasm_score_helinivan'] == 'derison') | (merged_pd['sarcasm_score_helinivan'] == 'normal')]
merged_pd = merged_pd[(merged_pd['sarcasm_prediction_t5'] == 'derison') | (merged_pd['sarcasm_prediction_t5'] == 'normal')]

In [80]:
merged_pd.head(5)

,review_id,sarcasm_score_sdcr,sarcasm_score_helinivan,sarcasm_prediction_t5,app_id,app_name,language,review,timestamp_created,timestamp_updated,...,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,publisher,genres,notes,cleaned_review,token_length
0,70165190,normal,normal,normal,107410,Arma 3,english,Extremely in depth play for military tactics. ...,2020-06-01 21:45:47,2020-06-01 21:45:47,...,1,3388.0000,0.0000,343.0000,2020-08-26 03:57:06,None,None,None,extremely in depth play for military tactics. ...,64
1,85003471,normal,normal,normal,1145360,Hades,english,A fantastic rouge-lite that has enough variety...,2021-01-19 23:42:57,2021-01-19 23:42:57,...,1,5352.0000,6.0000,5351.0000,2021-01-20 00:11:54,Supergiant Games,"[Action, Indie, RPG]",None,a fantastic rouge-lite that has enough variety...,59
2,79658641,normal,normal,normal,683320,GRIS,english,"I like artsy little indy games, and this one t...",2020-11-22 13:11:17,2020-11-22 13:11:17,...,67,84.0000,0.0000,84.0000,2020-11-22 12:58:52,Devolver Digital,"[Adventure, Indie]",None,"i like artsy little indy games, and this one t...",262
3,61385460,normal,normal,normal,814380,Sekiro™: Shadows Die Twice,english,hard game veri good cool animations i lika ver...,2020-01-05 22:43:23,2020-01-05 22:43:23,...,2,1871.0000,0.0000,1169.0000,2020-11-19 11:57:47,Activision (Excluding Japan and Asia),"[Action, Adventure]",None,hard game veri good cool animations i lika ver...,35
4,64935286,normal,derison,derison,374320,DARK SOULS™ III,english,"very good, anyone who say its bad says so beca...",2020-03-12 23:31:19,2020-03-12 23:31:19,...,11,9444.0000,0.0000,4652.0000,2020-12-08 16:38:00,"FromSoftware, Inc.",[Action],None,"very good, anyone who say its bad says so beca...",17


In [87]:
# funktion um die ergebnisse zu vergleichen
def detailed_derison_recommendation_comparison(df):
    # Gesamtzahl Reviews
    total_reviews = len(df)

    # Übereinstimmungen zwischen den drei Modellen
    df['all_agree'] = (df['sarcasm_score_sdcr'] == df['sarcasm_score_helinivan']) & \
                      (df['sarcasm_score_helinivan'] == df['sarcasm_prediction_t5'])

    # mind ein Modell mit derison
    df['positive_derison'] = ((df['sarcasm_score_sdcr'] == 'derison') | 
                              (df['sarcasm_score_helinivan'] == 'derison') | 
                              (df['sarcasm_prediction_t5'] == 'derison')) & ~df['all_agree']

    df['negative_derison'] = ((df['sarcasm_score_sdcr'] == 'normal') | 
                              (df['sarcasm_score_helinivan'] == 'normal') | 
                              (df['sarcasm_prediction_t5'] == 'normal')) & ~df['all_agree']

    # Berechnung der Derison- und Normal-Vorhersagen pro Modell und Empfehlung
    def calculate_recommendation_stats(df, model_col, model_name):
        recommended = df[df['recommended'] == True]
        not_recommended = df[df['recommended'] == False]

        derison_recommended = recommended[recommended[model_col] == 'derison'].shape[0]
        normal_recommended = recommended[recommended[model_col] == 'normal'].shape[0]

        derison_not_recommended = not_recommended[not_recommended[model_col] == 'derison'].shape[0]
        normal_not_recommended = not_recommended[not_recommended[model_col] == 'normal'].shape[0]

        print(f"\nModell: {model_name}")
        print(f"Empfohlene Spiele (recommended=True):")
        print(f"  - Derison: {derison_recommended}")
        print(f"  - Normal: {normal_recommended}")
        print(f"Nicht empfohlene Spiele (recommended=False):")
        print(f"  - Derison: {derison_not_recommended}")
        print(f"  - Normal: {normal_not_recommended}")

    # durchlaufen für alle modelle
    calculate_recommendation_stats(df, 'sarcasm_score_sdcr', 'dnzblgn/Sarcasm-Detection-Customer-Reviews')
    calculate_recommendation_stats(df, 'sarcasm_score_helinivan', 'helinivan/english-sarcasm-detector')
    calculate_recommendation_stats(df, 'sarcasm_prediction_t5', 'mrm8488/t5-base-finetuned-sarcasm-twitter')

#ausführen
detailed_derison_recommendation_comparison(merged_pd)


Modell: dnzblgn/Sarcasm-Detection-Customer-Reviews
Empfohlene Spiele (recommended=True):
  - Derison: 202
  - Normal: 27688
Nicht empfohlene Spiele (recommended=False):
  - Derison: 70
  - Normal: 7842

Modell: helinivan/english-sarcasm-detector
Empfohlene Spiele (recommended=True):
  - Derison: 1270
  - Normal: 26620
Nicht empfohlene Spiele (recommended=False):
  - Derison: 451
  - Normal: 7461

Modell: mrm8488/t5-base-finetuned-sarcasm-twitter
Empfohlene Spiele (recommended=True):
  - Derison: 5226
  - Normal: 22664
Nicht empfohlene Spiele (recommended=False):
  - Derison: 3544
  - Normal: 4368


Wir sehen, das Sarkasmus kein richtiges Muster für emphelungsmuster gibt. Dies könnte davon abhängen, dass die vermutung von Sarkasmus nicht nicht auf eine Empfehlung basieren muss. Dies und die Tatsache, dass die Modelle nicht gut auf Steam Reviews trainiert sind, könnte der Grund sein, warum die Modelle nicht gut funktionieren.

# Zusammenfassung der Ergebnisse

In diesem Notebook haben wir drei verschiedene Modelle zur Sarkasmuserkennung auf Steam Reviews getestet. Die Modelle wurden auf unterschiedlichen Datensätzen trainiert und zeigen unterschiedliche Ergebnisse in der Erkennung von Sarkasmus in den Reviews. Hier sind die wichtigsten Erkenntnisse:

## Modelle und Ergebnisse

### 1. dnzblgn/Sarcasm-Detection-Customer-Reviews
- **Ergebnisse**: Das Modell konnte einige sarkastische Reviews erkennen, zeigte jedoch insgesamt eine geringe Genauigkeit bei der Erkennung von Sarkasmus in Steam Reviews.
- **Begründung**: Das Modell wurde auf allgemeinen Kundenrezensionen trainiert, die sich in Sprache und Stil stark von Steam Reviews unterscheiden.

### 2. helinivan/english-sarcasm-detector
- **Ergebnisse**: Das Modell zeigte eine noch geringere Genauigkeit bei der Erkennung von Sarkasmus in Steam Reviews im Vergleich zum ersten Modell.
- **Begründung**: Das Modell wurde auf Nachrichtenüberschriften trainiert, die eine andere Struktur und einen anderen Ton als Steam Reviews haben.

### 3. mrm8488/t5-base-finetuned-sarcasm-twitter
- **Ergebnisse**: Das Modell zeigte ebenfalls eine geringe Genauigkeit bei der Erkennung von Sarkasmus in Steam Reviews.
- **Begründung**: Das Modell wurde auf Tweets trainiert, die in der Regel kürzer und informeller sind als Steam Reviews.

## Fazit
Keines der getesteten Modelle konnte zufriedenstellende Ergebnisse bei der Erkennung von Sarkasmus in Steam Reviews liefern. Die Hauptgründe dafür sind:

1. **Unterschiedliche Trainingsdatensätze**: Die Modelle wurden auf Datensätzen trainiert, die sich stark von Steam Reviews unterscheiden. Kundenrezensionen, Nachrichtenüberschriften und Tweets haben jeweils ihre eigenen sprachlichen und stilistischen Merkmale, die nicht direkt auf Steam Reviews übertragbar sind.

2. **Spezifität der Steam Reviews**: Steam Reviews enthalten oft spezifische Begriffe und Phrasen, die in den Trainingsdatensätzen der Modelle nicht vorkommen. Dies erschwert die Erkennung von Sarkasmus in diesem speziellen Kontext.

3. **Länge und Struktur der Reviews**: Steam Reviews sind oft länger und komplexer als die Texte, auf denen die Modelle trainiert wurden. Dies kann die Fähigkeit der Modelle, Sarkasmus zu erkennen, weiter beeinträchtigen.

Um die Erkennung von Sarkasmus in Steam Reviews zu verbessern, wäre es sinnvoll, ein Modell speziell auf einem Datensatz von Steam Reviews zu trainieren. Dies würde sicherstellen, dass das Modell die spezifischen sprachlichen und stilistischen Merkmale dieser Reviews besser versteht und somit genauere Ergebnisse liefert. Dies ist aber derzeit nicht möglich, da kein spezieller Datensatz für Steam Reviews existiert, welches diese als Sarkasmus oder nicht Sarkasmus markiert.

Dies wäre für eine zukünftige Arbeit ein interessanter Ansatz, um die Sentiment-Analyse von Steam Reviews weiter zu verbessern und Sarkasmus effektiver zu erkennen.